# EDA Exploration Notebook

Exploratory analysis and experimentation. Production logic is in `src/eda.py`.

## Setup

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["figure.dpi"] = 300

INCLUDE_SOFT_CHURN = False

current_dir = Path().resolve()
if (current_dir / "churn.duckdb").exists():
    PROJECT_ROOT = current_dir
elif (current_dir.parent / "churn.duckdb").exists():
    PROJECT_ROOT = current_dir.parent
else:
    PROJECT_ROOT = current_dir.parent

DB_PATH = PROJECT_ROOT / "churn.duckdb"
print(f"Database: {DB_PATH}")

## Load Data

In [ ]:
conn = duckdb.connect(DB_PATH.as_posix())
df_full = conn.execute("SELECT * FROM analytics.churn_features").df()
conn.close()

print(f"Shape: {df_full.shape[0]:,} rows × {df_full.shape[1]} columns")

df_full['lifecycle_stage'] = df_full['recency_days'].apply(
    lambda x: 'Active' if x <= 30 else ('At Risk (31-45d)' if x <= 45 else 'Churned')
)

df_lifecycle = df_full.copy()

if not INCLUDE_SOFT_CHURN:
    df = df_full[df_full['churn_label'].notna()].copy()
    print(f"Filtered to labeled customers only")
else:
    df = df_full.copy()

df_labeled = df.copy()

churned = df['churn_label'] == 1
active = df['churn_label'] == 0
churn_rate = churned.sum() / (churned.sum() + active.sum()) * 100 if (churned.sum() + active.sum()) > 0 else 0
print(f"Churn rate: {churn_rate:.2f}% ({churned.sum():,} churned / {active.sum():,} active)")

df['churn_status'] = df['churn_label'].map({1: 'Churned', 0: 'Active'})
df_full.head()

## Lifecycle Distribution

In [ ]:
lifecycle_counts = df_lifecycle['lifecycle_stage'].value_counts()
lifecycle_order = ['Active', 'At Risk (31-45d)', 'Churned']
lifecycle_counts = lifecycle_counts.reindex([s for s in lifecycle_order if s in lifecycle_counts.index])

print("Lifecycle stage counts:")
for stage in lifecycle_order:
    if stage in lifecycle_counts.index:
        count = lifecycle_counts[stage]
        pct = (count / len(df_lifecycle)) * 100
        print(f"   {stage}: {count:,} ({pct:.1f}%)")

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(
    data=df_lifecycle,
    x='lifecycle_stage',
    hue='lifecycle_stage',
    order=lifecycle_order,
    palette={'Active': 'green', 'At Risk (31-45d)': 'orange', 'Churned': 'red'},
    legend=False
)
plt.title('Customer Lifecycle Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Lifecycle Stage', fontsize=12)
plt.ylabel('Number of Customers', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Lifecycle by Plan

In [ ]:
lifecycle_by_plan = df_lifecycle.groupby(['plan', 'lifecycle_stage']).size().reset_index(name='count')
lifecycle_by_plan_pivot = lifecycle_by_plan.pivot(index='plan', columns='lifecycle_stage', values='count').fillna(0)
lifecycle_by_plan_pivot = lifecycle_by_plan_pivot.reindex(columns=[s for s in lifecycle_order if s in lifecycle_by_plan_pivot.columns])

lifecycle_by_plan_pivot

In [ ]:
fig = px.bar(
    lifecycle_by_plan,
    x='plan',
    y='count',
    color='lifecycle_stage',
    color_discrete_map={'Active': 'green', 'At Risk (31-45d)': 'orange', 'Churned': 'red'},
    title='Customer Lifecycle by Plan',
    labels={'count': 'Number of Customers', 'plan': 'Plan'},
    barmode='stack'
)
fig.update_layout(
    height=500,
    xaxis={'categoryorder': 'total descending'}
)
fig.show()

## Recency Density by Lifecycle

In [ ]:
plt.figure(figsize=(12, 6))
df_lifecycle_recency = df_lifecycle[df_lifecycle['recency_days'].notna()].copy()
if len(df_lifecycle_recency) > 0:
    sns.kdeplot(
        data=df_lifecycle_recency,
        x='recency_days',
        hue='lifecycle_stage',
        common_norm=False,
        palette={'Active': 'green', 'At Risk (31-45d)': 'orange', 'Churned': 'red'}
    )
else:
    plt.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=plt.gca().transAxes)

plt.title('Recency Density by Lifecycle Stage', fontsize=14, fontweight='bold')
plt.xlabel('Recency (days)', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Overview Statistics

In [ ]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'customer_id']

df[numeric_cols].describe().round(2)

In [ ]:
print(f"Recency days: {df['recency_days'].median():.1f}")
print(f"Usage trend 30d: {df['usage_trend_30d'].median():.2f}")
print(f"Total payment issues: {df['total_payment_issues'].median():.0f}")
print(f"Total tickets: {df['total_tickets'].median():.0f}")

## Churn by Plan

In [ ]:
churn_by_plan = df.groupby('plan').agg({
    'churn_label': lambda x: ((x == 1).sum() / ((x == 1).sum() + (x == 0).sum()) * 100) if ((x == 1).sum() + (x == 0).sum()) > 0 else 0,
    'customer_id': 'count'
}).rename(columns={'churn_label': 'churn_rate', 'customer_id': 'total_customers'})
churn_by_plan = churn_by_plan.sort_values('churn_rate', ascending=False)

churn_by_plan

In [ ]:
fig = px.bar(
    churn_by_plan.reset_index(),
    x='plan',
    y='churn_rate',
    title='Churn Rate by Plan',
    labels={'churn_rate': 'Churn Rate (%)', 'plan': 'Plan'},
    text='churn_rate',
    color='churn_rate',
    color_continuous_scale='Reds'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    showlegend=False,
    height=500,
    xaxis={'categoryorder': 'total descending'}
)
fig.show()

## Churn by Region

In [ ]:
churn_by_region = df.groupby('region').agg({
    'churn_label': lambda x: ((x == 1).sum() / ((x == 1).sum() + (x == 0).sum()) * 100) if ((x == 1).sum() + (x == 0).sum()) > 0 else 0,
    'customer_id': 'count'
}).rename(columns={'churn_label': 'churn_rate', 'customer_id': 'total_customers'})
churn_by_region = churn_by_region.sort_values('churn_rate', ascending=False)

churn_by_region

In [ ]:
fig = px.bar(
    churn_by_region.reset_index(),
    x='region',
    y='churn_rate',
    title='Churn Rate by Region',
    labels={'churn_rate': 'Churn Rate (%)', 'region': 'Region'},
    text='churn_rate',
    color='churn_rate',
    color_continuous_scale='Oranges'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    showlegend=False,
    height=500,
    xaxis={'categoryorder': 'total descending'}
)
fig.show()

## Usage Trend Distribution

In [ ]:
active_median = df[df['churn_label'] == 0]['usage_trend_30d'].median()
churned_median = df[df['churn_label'] == 1]['usage_trend_30d'].median()
print(f"Active median: {active_median:.2f}" if not pd.isna(active_median) else "Active median: N/A")
print(f"Churned median: {churned_median:.2f}" if not pd.isna(churned_median) else "Churned median: N/A")

In [ ]:
plt.figure(figsize=(12, 6))
df_usage = df[df['usage_trend_30d'].notna()].copy()
if len(df_usage) > 0:
    has_both_statuses = len(df_usage['churn_status'].unique()) > 1
    use_kde = len(df_usage) >= 100 and has_both_statuses
    
    if has_both_statuses:
        sns.histplot(
            data=df_usage,
            x='usage_trend_30d',
            hue='churn_status',
            bins=50,
            kde=use_kde,
            alpha=0.6,
            palette={'Active': 'green', 'Churned': 'red'}
        )
    else:
        sns.histplot(
            data=df_usage,
            x='usage_trend_30d',
            bins=50,
            kde=use_kde,
            alpha=0.6,
            color='red' if df_usage['churn_status'].iloc[0] == 'Churned' else 'green'
        )
else:
    plt.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=plt.gca().transAxes)

plt.title('Usage Trend (30d) Distribution by Churn Status', fontsize=14, fontweight='bold')
plt.xlabel('Usage Trend 30d (minutes)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
if len(df_usage) > 0 and len(df_usage['churn_status'].unique()) > 1:
    plt.legend(title='Churn Status')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Recency Distribution

In [ ]:
active_recency = df[df['churn_label'] == 0]['recency_days'].median()
churned_recency = df[df['churn_label'] == 1]['recency_days'].median()
print(f"Active median recency: {active_recency:.1f} days" if not pd.isna(active_recency) else "Active median recency: N/A")
print(f"Churned median recency: {churned_recency:.1f} days" if not pd.isna(churned_recency) else "Churned median recency: N/A")

In [ ]:
plt.figure(figsize=(10, 6))
has_both_statuses = len(df['churn_status'].unique()) > 1
if has_both_statuses:
    sns.boxplot(
        data=df,
        x='churn_status',
        y='recency_days',
        palette={'Active': 'green', 'Churned': 'red'}
    )
else:
    status = df['churn_status'].iloc[0]
    color = 'red' if status == 'Churned' else 'green'
    sns.boxplot(
        data=df,
        y='recency_days',
        color=color
    )
    plt.xlabel('Churn Status', fontsize=12)
    plt.xticks([0], [status])
plt.title('Recency Distribution by Churn Status', fontsize=14, fontweight='bold')
if has_both_statuses:
    plt.xlabel('Churn Status', fontsize=12)
plt.ylabel('Recency (days)', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Retention Funnel

In [ ]:
active_count = len(df_lifecycle[df_lifecycle['lifecycle_stage'] == 'Active'])
at_risk_count = len(df_lifecycle[df_lifecycle['lifecycle_stage'] == 'At Risk (31-45d)'])
churned_count = len(df_lifecycle[df_lifecycle['lifecycle_stage'] == 'Churned'])
total_customers = len(df_lifecycle)

funnel_data = pd.DataFrame({
    'Stage': ['Total Customers', 'Active (<=30 days)', 'At Risk (31-45 days)', 'Churned (>45 days)'],
    'Count': [total_customers, active_count, at_risk_count, churned_count]
})

funnel_data

In [ ]:
fig = go.Figure(go.Funnel(
    y=funnel_data['Stage'],
    x=funnel_data['Count'],
    textposition="inside",
    textinfo="value+percent initial",
    marker={"color": ["#1f77b4", "#2ca02c", "#ff7f0e", "#d62728"]}
))
fig.update_layout(
    title='Customer Retention Funnel',
    height=500
)
fig.show()

## Support Load Distribution

In [ ]:
active_tickets_med = df[df['churn_label'] == 0]['total_tickets'].median()
churned_tickets_med = df[df['churn_label'] == 1]['total_tickets'].median()
print(f"Active median tickets: {active_tickets_med:.0f}" if not pd.isna(active_tickets_med) else "Active median tickets: N/A")
print(f"Churned median tickets: {churned_tickets_med:.0f}" if not pd.isna(churned_tickets_med) else "Churned median tickets: N/A")

In [ ]:
plt.figure(figsize=(10, 6))
has_both_statuses = len(df['churn_status'].unique()) > 1
if has_both_statuses:
    sns.boxplot(
        data=df,
        x='churn_status',
        y='total_tickets',
        palette={'Active': 'green', 'Churned': 'red'}
    )
else:
    status = df['churn_status'].iloc[0]
    color = 'red' if status == 'Churned' else 'green'
    sns.boxplot(
        data=df,
        y='total_tickets',
        color=color
    )
    plt.xticks([0], [status])
plt.title('Support Ticket Distribution by Churn Status', fontsize=14, fontweight='bold')
plt.xlabel('Churn Status', fontsize=12)
plt.ylabel('Total Tickets', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Key Insights

In [ ]:
insights = []

highest_plan_churn = churn_by_plan.index[0]
highest_plan_rate = churn_by_plan.loc[highest_plan_churn, 'churn_rate']
lowest_plan_churn = churn_by_plan.index[-1]
lowest_plan_rate = churn_by_plan.loc[lowest_plan_churn, 'churn_rate']
insights.append(f"Plan '{highest_plan_churn}' has highest churn rate ({highest_plan_rate:.1f}%), '{lowest_plan_churn}' has lowest ({lowest_plan_rate:.1f}%)")

highest_region_churn = churn_by_region.index[0]
highest_region_rate = churn_by_region.loc[highest_region_churn, 'churn_rate']
insights.append(f"Region '{highest_region_churn}' shows highest churn rate ({highest_region_rate:.1f}%)")

at_risk_pct = (at_risk_count / total_customers) * 100
insights.append(f"{at_risk_pct:.1f}% of customers are in At Risk (31-45d) stage, representing a key intervention opportunity")

at_risk_by_plan = df_lifecycle[df_lifecycle['lifecycle_stage'] == 'At Risk (31-45d)'].groupby('plan').size()
if len(at_risk_by_plan) > 0:
    highest_at_risk_plan = at_risk_by_plan.idxmax()
    highest_at_risk_count = at_risk_by_plan.max()
    plan_total = df_lifecycle[df_lifecycle['plan'] == highest_at_risk_plan].shape[0]
    highest_at_risk_pct = (highest_at_risk_count / plan_total) * 100 if plan_total > 0 else 0
    insights.append(f"Plan '{highest_at_risk_plan}' has the highest concentration of At Risk customers ({highest_at_risk_pct:.1f}% of plan customers)")

active_trend = df[df['churn_label'] == 0]['usage_trend_30d'].median()
churned_trend = df[df['churn_label'] == 1]['usage_trend_30d'].median()
insights.append(f"Churned customers show {churned_trend:.2f} min/day usage trend vs {active_trend:.2f} for active (median)")

active_tickets = df[df['churn_label'] == 0]['total_tickets'].median()
churned_tickets = df[df['churn_label'] == 1]['total_tickets'].median()
insights.append(f"Churned customers have median {churned_tickets:.0f} support tickets vs {active_tickets:.0f} for active")

insights.append(f"Overall churn rate: {churn_rate:.2f}% ({churned.sum():,} churned / {active.sum():,} active)")

high_recency = df[df['churn_label'] == 1]['recency_days'].median()
low_recency = df[df['churn_label'] == 0]['recency_days'].median()
insights.append(f"Churned customers have median recency of {high_recency:.0f} days vs {low_recency:.0f} days for active")

for insight in insights:
    print(insight)